In [79]:
# import tensorflow and numpy
import tensorflow as tf
import numpy as np

In [140]:
# setup the parameters
# number of different M's
gradients = 5
# number of input values
vals = 3
iterations = 5000
learning_rate = 0.01

# maximum answer to be learnt
max = gradients**2 + vals

# defining function to make training data
def training_data():
    
    n = 0
    rows = 1
    # array which we are using as our x values 
    # in equation of linear line, y = Mx + b
    # it includes 1 extra value as this will be used as our labal
    x = np.arange(vals+1).astype(np.int32)
    # empty array to write our training data to
    y = np.array([])
    
    # loop so it cycles through every gradient and adds on different b's
    for i in range(gradients):
        # we loop it so it also creates lines with b's
        # the maximum values will always be gradients^2
        for e in range(max-vals*i):
            
            # append to y, x*n + e, where n = M and e = b
            # reshape so that each time it appends it creates a new line
            y = np.append(y, x*n+e).reshape(rows,vals+1)
            
            # increase number of rows to reshape it
            rows += 1
        
        # increase gradient by 1
        n+=1
        
    # return the training data
    # and number of lines to learn
    return(y,np.size(y,0))

# print the training data
print(training_data())

(array([[  0.,   0.,   0.,   0.],
       [  1.,   1.,   1.,   1.],
       [  2.,   2.,   2.,   2.],
       [  3.,   3.,   3.,   3.],
       [  4.,   4.,   4.,   4.],
       [  5.,   5.,   5.,   5.],
       [  6.,   6.,   6.,   6.],
       [  7.,   7.,   7.,   7.],
       [  8.,   8.,   8.,   8.],
       [  9.,   9.,   9.,   9.],
       [ 10.,  10.,  10.,  10.],
       [ 11.,  11.,  11.,  11.],
       [ 12.,  12.,  12.,  12.],
       [ 13.,  13.,  13.,  13.],
       [ 14.,  14.,  14.,  14.],
       [ 15.,  15.,  15.,  15.],
       [ 16.,  16.,  16.,  16.],
       [ 17.,  17.,  17.,  17.],
       [ 18.,  18.,  18.,  18.],
       [ 19.,  19.,  19.,  19.],
       [ 20.,  20.,  20.,  20.],
       [ 21.,  21.,  21.,  21.],
       [ 22.,  22.,  22.,  22.],
       [ 23.,  23.,  23.,  23.],
       [ 24.,  24.,  24.,  24.],
       [ 25.,  25.,  25.,  25.],
       [ 26.,  26.,  26.,  26.],
       [ 27.,  27.,  27.,  27.],
       [  0.,   1.,   2.,   3.],
       [  1.,   2.,   3.,   4.],
       [ 

In [141]:
# data = training data
# training lines = number of different lines
data,training_lines = training_data()

# the length is for when we convert the numbers into a binary array
# the array will be all zeros except one, which will be 1
# this will be the particle in this point in time
# each one is like a frame in a video
# this value is the size of the maximum value + 1
length = max+1

# the full length is the length of all the input frames stacked into one, 1d array
full_length = length*vals

# this function turns the data into the arrays explained above
def set_data():
    
    # there are two arrays, one for training data nad one for labels
    in_data = np.zeros([training_lines,vals,length])
    # the labels will be one-hot arrays
    lab = np.zeros([training_lines,1,length])
    
    # this sets the values specified in the training data to one
    for i in range(training_lines):
        # we need to set each individual input value
        for a in range(vals):
            # set the value to a 1
            in_data[i][a][data[i][a]] = 1
            
        # set the label value to a 1
        lab[i][0][data[i][vals]] = 1
        
    # here, we reshape it tto the full length 1d array
    in_data = in_data.reshape(training_lines,1,full_length)
    
    # return the data and labels
    return(in_data,lab)    

# print converted data
#print(set_data())

In [142]:
# we define the weights, biases and inputs
# this will be input training data
x = tf.placeholder(tf.float32, [None, full_length])
# weights and biases
W = tf.Variable(tf.zeros([full_length, length]))
b = tf.Variable(tf.zeros([length]))
# function which gives the out put of training data
y = tf.matmul(x, W) + b

# we will feed the labels in here
y_ = tf.placeholder(tf.float32, [None, length])

# configure the loss function, using cross entropy
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
# define the optimizer
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

In [143]:
# I am using a GPU
# this line limits memory usage of the GPU to 0.4 when session is created
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.4)

# create interactive session using the GPU line for above
sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options)) 

# initialize variables
init = tf.initialize_all_variables()
sess.run(init)

# now we run the learning loop
for _ in range(iterations):

    # set training data and labels
    x_data, y_data = set_data()
    # only use one lines data
    # chooses line data by evenly spreading data over iterations
    x_data = x_data[_//(iterations/(training_lines-1.))]
    y_data = y_data[_//(iterations/(training_lines-1.))]
    
    # run the training optimizer
    sess.run(train_step, feed_dict={x: x_data, y_: y_data})

    # print steps and error 20 times in total
    if _ % (iterations/20) == 0:
        print 'step', _, 'out of', iterations
        print 'error =', sess.run(cross_entropy, feed_dict={x: x_data, y_: y_data})

Exception AssertionError: AssertionError("Nesting violated for default stack of <type 'weakref'> objects",) in <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x7f953c149e50>> ignored
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:28: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:31: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:19: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:20: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


step 0 out of 5000
error = 3.3287
step 250 out of 5000
error = 2.66426
step 500 out of 5000
error = 2.01764
step 750 out of 5000
error = 2.94874
step 1000 out of 5000
error = 2.28719
step 1250 out of 5000
error = 3.24628
step 1500 out of 5000
error = 2.31289
step 1750 out of 5000
error = 3.30035
step 2000 out of 5000
error = 2.55231
step 2250 out of 5000
error = 3.52671
step 2500 out of 5000
error = 2.64508
step 2750 out of 5000
error = 1.98752
step 3000 out of 5000
error = 2.87051
step 3250 out of 5000
error = 2.18571
step 3500 out of 5000
error = 2.96716
step 3750 out of 5000
error = 2.29359
step 4000 out of 5000
error = 3.22008
step 4250 out of 5000
error = 2.44035
step 4500 out of 5000
error = 3.35377
step 4750 out of 5000
error = 2.59095


In [148]:
# initialize array to record test values
test_line = np.array([])

print 'input values, the max answer is', max-1, 'and max gradient is', gradients-1
# loop to ask user for input values
for i in range(vals):
    print '\nvalue',i+1,'?'
    val = int(input())
    test_line = np.append(test_line, val)
    
# define function to convert test data into shape and form
# of training data
def test_model():
    
    # this is the array we are writing to
    input_array = np.array([])
    
    # for each input value, write to the input array
    # with a 1 in the position
    for a in range(vals):
        test_array = np.zeros([1,length])
        test_array[0][test_line[a]] = 1
        # we write to array and shape so new line for each test value
        input_array = np.append(input_array, test_array).reshape([1,(a+1)*length])
        
    # return array
    return(input_array)   

# print array
#test_model()

input values, the max answer is 27 and max gradient is 4

value 1 ?
6

value 2 ?
8

value 3 ?
10


In [149]:
# run the function to get output
# using new weights nad biases
probs = sess.run(y, feed_dict={x:test_model()})

# print the probabilities
print(probs)

# print learnt answer and it probability
print '\nLearnt answer =', np.argmax(probs)
print 'Probaility of', np.argmax(probs), '=', probs[0][np.argmax(probs)]

# "true" answer, should work with all linear lines
answer = (test_line[1]-test_line[0])+test_line[vals-1]

# print true answer and its probabilities
print '\nNumerical answer =', answer
print 'Probability of', answer, '=', probs[0][answer]

[[-0.90991259 -0.91085827 -0.91804415 -0.75601995 -0.77218199 -0.77532107
  -0.17859304 -0.61192977 -0.2206229   0.03747336  0.41671541  0.02512801
   1.22958863  0.42141163  0.89484167  0.50177759  0.52957666  0.1166441
   0.59160638  0.16506003  0.18949726  0.21969202  0.25188628  0.28090489
   0.33168629  0.39498532  0.44685078  0.03957763 -1.03141916]]

Learnt answer = 12
Probaility of 12 = 1.22959

Numerical answer = 12.0
Probability of 12.0 = 1.22959


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:22: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:17: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [108]:
print 40000//(iterations/(training_lines-0.))

895.0
